In [7]:
# HIDE
import pandas as pd
import numpy as np
np.random.seed(7)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", palette="muted", color_codes=True, context="talk")
from IPython import display
%matplotlib inline

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import keras as ke
import keras.backend as K
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.models import load_model


create_gif = False

# model reconstruction from JSON:
from keras.models import model_from_json

print(f"sklearn: {sk.__version__}")
print(f"pandas: {pd.__version__}")
print(f"kerads: {ke.__version__}")


import my_functions_product1 as myFC
import matplotlib.pyplot as plt

sklearn: 0.19.1
pandas: 0.23.0
kerads: 2.1.6


In [15]:
# reading the file from a path and list the field names and input columns of interest

path = 'data/adult.data'
output_path = 'Feature-Datasets/adult/'

#path = 'data/bank.csv'
#output_path = 'Feature-Datasets/bank/'


X_df, Ybin, Zbin, result_fname = myFC.read_process_data_output_bias(path)

Columns in your data are:
  ['39' ' State-gov' ' 77516' ' Bachelors' ' 13' ' Never-married'
 ' Adm-clerical' ' Not-in-family' ' White' ' Male' ' 2174' ' 0' ' 40'
 ' United-States' ' <=50K']
Your possible target columns '{' Male', ' 77516', ' State-gov', ' Bachelors', ' Adm-clerical', ' 40', ' White', ' 2174', ' 0', ' United-States', ' <=50K', '39', ' 13', ' Never-married', ' Not-in-family'}'
Which column is your target column?: <=50K
Your target column is ' <=50K'
-----------------------------------

Do you have any column for your current prediction? Please enter Y or N:N
Do you have any column for your current prediction? You entered'N'
-----------------------------------

Your prediction column possible values '{' >50K', ' <=50K'}'
Please enter two different labels in the target field: 
Enter label 0: <=50K
Your target label 0 is ' <=50K'
-----------------------------------

Enter label 1: >50K
Your target label 1 is ' >50K'
-----------------------------------

Your possible Sensiti

### Feature Creation

In [16]:
X_train, X_test, y_train, y_test, Z_train, Z_test = myFC.feature_creation(X_df, Ybin, Zbin)

(16280, 95)
(16280, 95)


In [17]:
feature_train_df = X_train
feature_train_df['y'] = y_train
feature_train_df['Z'] = Z_train
feature_train_df.to_csv(output_path+'train'+'.csv', index=False)

feature_test_df = X_test
feature_test_df['y'] = y_test
feature_test_df['Z'] = Z_test
feature_test_df.to_csv(output_path+'test'+'.csv', index=False)




### Setting up architectures

In [18]:
#==============================================================================
def User_Classifier_arch(n_features):
    inputs = Input(shape=(n_features,))
    dense1 = Dense(32, activation='relu')(inputs)
    dropout1 = Dropout(0.2)(dense1)
    dense2 = Dense(32, activation='relu')(dropout1)
    dropout2 = Dropout(0.2)(dense2)
    dense3 = Dense(32, activation="relu")(dropout2)
    dropout3 = Dropout(0.2)(dense3)
    dense4 = Dense(32, activation="relu")(dropout3)
    dropout4 = Dropout(0.2)(dense4)
    outputs = Dense(1, activation='sigmoid')(dropout4)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model    
#==============================================================================
def User_Adversary_arch(inputs):
    dense1 = Dense(32, activation='relu')(inputs)
    dense2 = Dense(32, activation='relu')(dense1)
    dense3 = Dense(32, activation='relu')(dense2)
    dense4 = Dense(32, activation="relu")(dense3)
    outputs = Dense(1, activation='sigmoid')(dense4)
    return Model(inputs=[inputs], outputs=outputs)   
#==============================================================================
def User_main_task_adv_architecture(n_features):
    main_task_arch = User_Classifier_arch(n_features)
    main_task_arch_json_string = main_task_arch.to_json()
    adv_inputs = Input(shape=(1,))
    adv_task_arch = User_Adversary_arch(adv_inputs)
    adv_task_arch_json_string = adv_task_arch.to_json()
    return main_task_arch_json_string, adv_task_arch_json_string  

## Saving architectures

In [19]:
n_features = X_train.shape[1]
main_task_arch_json_string, adv_task_arch_json_string = User_main_task_adv_architecture(n_features)

main_task_arch_json_string_file = output_path+'main_task_arch_json_string_file'+'.txt'
with open(main_task_arch_json_string_file,"wt") as out_file:
    out_file.write(main_task_arch_json_string)
    
adv_task_arch_json_string_file = output_path+'adv_task_arch_json_string_file'+'.txt'
with open(adv_task_arch_json_string_file,"wt") as out_file:
    out_file.write(adv_task_arch_json_string)    

In [20]:
"""# Write a file
with open("test.txt", "wt") as out_file:
    out_file.write("This Text is going to out file\nLook at it and see!")

# Read a file
with open("test.txt", "rt") as in_file:
    text = in_file.read()

print(text)"""

'# Write a file\nwith open("test.txt", "wt") as out_file:\n    out_file.write("This Text is going to out file\nLook at it and see!")\n\n# Read a file\nwith open("test.txt", "rt") as in_file:\n    text = in_file.read()\n\nprint(text)'

## Model and compile only main task: check prediction results

In [21]:
h5_file_name = output_path+'main_task_ori_trained_model.h5'
main_task_accuracy, p_rule_for_Y1, y_pred = myFC.pre_train_main_task(main_task_arch_json_string, h5_file_name,X_train, y_train,X_test,y_test,Z_test)

Accuracy: 99.93
